In [1]:
class Card(object):
    """ A playing card. """
    RANKS = ["A", "2", "3", "4", "5", "6", "7","8", "9", "10", "J", "Q", "K"]
    SUITS = ["C", "D", "H", "S"]

    def __init__(self, rank, suit):
        self.rank = rank
        self.suit = suit

    def __str__(self):
        rep = self.rank + ' of ' + self.suit
        return rep

In [2]:
class Hand(object):
    """ A hand of playing cards. """
    def __init__(self):
        self.cards = []

    def __str__(self):
        if self.cards:
           rep = ""
           for card in self.cards:
               rep += str(card) + "\t"
        else:
            rep = "<empty>"
        return rep

    def clear(self):
        self.cards = []

    def add(self, card):
        self.cards.append(card)

    def give(self, card, other_hand):
        self.cards.remove(card)
        other_hand.add(card)

In [3]:
class Deck(Hand):
    """ A deck of playing cards. """
    def populate(self):
        if len(self.cards) < 52:
            
            for suit in Card.SUITS:
                for rank in Card.RANKS: 
                    self.add(Card(rank, suit))
        else:
            print('Deck full!')

    def shuffle(self,ntimes = 5):
        import random
        for times in range(ntimes):
            random.shuffle(self.cards)

    def deal(self, hands, per_hand = 1):
        for rounds in range(per_hand):
            for hand in hands:
                if self.cards:
                    top_card = self.cards[0]
                    self.give(top_card, hand)
                else:
                    print("Can't continue deal. Out of cards!")

In [4]:
class BJ_Card(Card):
    """ A Blackjack Card. """
    ACE_VALUE = 1
    
    def value(self):
        v = BJ_Card.RANKS.index(self.rank) + 1
        if v < 10:
            v = v
        else:
            v = 10
        return v

In [5]:
card1 = BJ_Card('10','C')

In [6]:
card1.value()

10

In [7]:
class BJ_Hand(Hand):
       
    def total(self):
        
        for card in self.cards:
            if not card.value():
                return None
        
        t = 0
        for card in self.cards:
              t += card.value()

        contains_ace = False
        for card in self.cards:
            if card.value() == card.ACE_VALUE:
                contains_ace = True
                
        if contains_ace and t <= 11:
            t += 10            
        
        return t

    def is_busted(self):
        return self.total() > 21


In [8]:
class BJ_Deck(Deck):
    """ A Blackjack Deck. """
    def populate(self):
        for suit in BJ_Card.SUITS: 
            for rank in BJ_Card.RANKS: 
                self.cards.append(BJ_Card(rank, suit))

In [9]:
class BJ_Dealer(BJ_Hand):
    """ A Blackjack Dealer. """
    def is_hitting(self):
        return self.total < 17

    def bust(self):
        print(self.name, "busts.")

In [28]:
class BJ_credit(object):
    def __init__(self,credit):
        self.credit = credit
        
    def win(self,bet):
        self.credit = self.credit + bet
        return self.credit
        
    def lose(self,bet):
        self.credit = self.credit - bet
        return self.credit

In [41]:
cred = BJ_credit(1000)
cred.win(10)
print(cred.credit)

1010


In [11]:
def reset_hands(hands):
    for hand in hands:
        hand.clear()

In [12]:
def print_hands(hands):
    print('playerhand: ',hands[0])
    print('dealerhand: ',hands[1])

In [45]:
def BJgame():
    from IPython.display import clear_output
    clear_output()
    print('Welcome to BlackJack!')
    deck = BJ_Deck()
    deck.populate()
    deck.shuffle()
    playerhand = BJ_Hand()
    dealerhand = BJ_Dealer()
    hands = [playerhand,dealerhand]
    reset_hands(hands)
    playercredit = BJ_credit(int(input('How much you want to play? ')))
    print_hands(hands)
    print('player credit: ',playercredit.credit)
    
    while playercredit.credit != 0:
    
        hand1 = [playerhand]
        deck.deal(hand1,2)
        clear_output()
        print_hands(hands)
        playerhand.total()
        print('player credit: ',playercredit.credit)
        bet = int(input('How much do you want to bet? :'))
        if input('Do you want to hit? y/n ') == 'y':
            deck.deal(hand1)
            clear_output()
            print_hands(hands)
            playerhand.total()
            if playerhand.is_busted():
            
                print('Bust!')
            print('player credit: ',playercredit.credit)
        elif playerhand.total() == 21:
        
            print('BLACKJACK!')
            print('player credit: ',playercredit.credit)
        
        hand2 = [dealerhand]
        deck.deal(hand2,2)
        clear_output()
        print_hands(hands)
        print('player credit: ',playercredit.credit)
        if dealerhand.total() < 17:
            deck.deal(hand2)
            clear_output()
            print_hands(hands)
            print('player credit: ',playercredit.credit)
            if dealerhand.total() > 21:
            
                print('Dealer Bust!')
                print('Player wins!')
            
        if playerhand.total() < dealerhand.total() and dealerhand.total() <= 21:
            playercredit.lose(bet)
            clear_output()
            print_hands(hands)
            print('player credit: ',playercredit.credit)
            print('player lose')
            
        elif playerhand.total() > 21:
            playercredit.lose(bet)
            clear_output()
            print_hands(hands)
            print('player credit: ',playercredit.credit)
            print('player lose')
            
            
        elif playerhand.total() > dealerhand.total() and playerhand.total() <= 21:
            playercredit.win(bet)
            clear_output()
            print_hands(hands)
            print('player credit: ',playercredit.credit)
            print('player win')
            
            
        if input('Continue? y/n : ') == 'y':
            reset_hands(hands)
        
        elif playercredit.credit == 0:
            print('Game over!')
            break
        else:
            break
        

In [44]:
BJgame()

playerhand:  10 of D	7 of H	K of C	
dealerhand:  10 of S	7 of S	
player credit:  0
player lose
Continue? y/n : y
Game over!
